<a href="https://colab.research.google.com/github/Alinehbg/CRP/blob/Joshua_Trial/Extract%20Names/Extract_Wiki_Data_names_try_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
names = pd.read_excel('/content/drive/MyDrive/CRP/Datasets/Dataset.xlsx')
names['Name'] = names.apply (lambda x: x['Name'].replace(".","") , axis = 1)
names['Name_Length'] = names.apply(lambda x: len(x['Name'].split(" ")), axis = 1)
names['First_Name_Length'] = names.apply(lambda x: len(x['Name'].split(" ")[0]), axis = 1)
names = names.loc[names['Name_Length'] == 2]
names = names.loc[names['First_Name_Length'] > 1]
names['Name'] = names.apply(lambda x: x['Name'].title(), axis = 1 )
names = names[["Name"]]
names_list = list(names['Name'])
names_list = list(set(names_list))
names.head()

,Name
6,John Aalberg
21,Andreea Aanei
22,Fritz Aanes
26,Johan Aantjes
27,Jan-Erik Aarberg


In [11]:
names.shape
# names.to_csv('/content/drive/MyDrive/CRP/Datasets/Names_List.csv')

(69441, 1)

#### functions 

In [27]:
def get_itemlabel(query): 
  name_list = []
  for x in query["results"]["bindings"]: 
    name_list.append(x["itemLabel"]["value"])
  return name_list

def retry(data): 
  response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  while response.status_code == 429:
    retry_after = response.headers.get("Retry-After")
    if retry_after:
      # If the value of "Retry-After" is a number, wait that many seconds
      if retry_after.isdigit():
        time.sleep(int(retry_after))
        # If the value of "Retry-After" is a date, parse it and wait until that time
      else:
        retry_time = datetime.datetime.strptime(retry_after, "%a, %d %b %Y %H:%M:%S GMT")
        if retry_time > datetime.datetime.now():
          time.sleep((retry_time - datetime.datetime.now()).total_seconds())
    # Try the request again
    response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  res = response.json()
  return res

def find_fname_id(name): 
  find_fn_id_query_male = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  find_fn_id_query_female = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query_male}
  res = retry(data)
  if res['results']['bindings'] == []: 
    data = {'query': find_fn_id_query_female}
    res = retry(data)
  try:
    id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
    #print(f"Cannot find first name id for {name}")
    id = []
  return id

def find_lname_id(name): 
  find_fn_id_query = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q101352.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query}
  res = retry(data)
  try:
      id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
      #print(f"Cannot find last name id for {name}")  
      id = []  
  return id

def find_similar_from_id(id): 
  #find similar names from the ID just found for the target name
  id = id
  find_similar_query = f"""
  SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    {{
      SELECT DISTINCT ?item WHERE {{
        ?item p:P460 ?statement0.
        ?statement0 (ps:P460/(wdt:P279*)) wd:{id}.
      }}
    }}
  }}
  """
  data = {'query': find_similar_query}
  res = retry(data)
  similar_name_list = get_itemlabel(res)
  return similar_name_list

def similar_full_names(name): 
  f_name, l_name = name.split(' ')

  f_name_id = find_fname_id(f_name)
  l_name_id = find_lname_id(l_name)

  if f_name_id == [] and l_name_id != []: 
    fname_similar = []
    lname_similar = find_similar_from_id(l_name_id)
    fname_list = []
    lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]
  elif l_name_id == [] and f_name_id != []: 
    lname_similar = []
    fname_similar = find_similar_from_id(f_name_id)
    lname_list = []
    fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
  elif f_name_id == [] and l_name_id == []: 
    fname_list = []
    lname_list = []
  else: 
    fname_similar = find_similar_from_id(f_name_id)
    lname_similar = find_similar_from_id(l_name_id)

    fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
    lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]
  
  similar_full_names = fname_list + lname_list
  pair = {name: similar_full_names}
  return pair

def Merge(dict1, dict2):
  return(dict1.update(dict2))

####requests and the rest

In [31]:
#testing merge function
name = 'Joshua Baker'
s = similar_full_names(name)
name1 = 'Jean Lahellec'
s1 = similar_full_names(name1)
Merge(s,s1)
len(s)

2

In [13]:
#convert Name column in dataframe to list
name_list = names['Name'].tolist()

In [ ]:
#loop through all the names in the list and add key(target name) value(list of similar names) pairs to dictionary 
name_dict = {}
for name in name_list: 
   temp = similar_full_names(name)
   Merge(name_dict, temp)

In [33]:
len(name_dict)

503

In [34]:
name_dict

{'John Aalberg': ['Juan Aalberg',
  'Ivo Aalberg',
  'Jack Aalberg',
  'Ivan Aalberg',
  'Ion Aalberg',
  'Gian Aalberg',
  'Giovanni Aalberg',
  'Hans Aalberg',
  'Jógvan Aalberg',
  'Johnny Aalberg',
  'Johannes Aalberg',
  'Ieuan Aalberg',
  'Jonas Aalberg',
  'Honza Aalberg',
  'Jean Aalberg',
  'Siôn Aalberg',
  'Yannis Aalberg',
  'Johan Aalberg',
  'Jan Aalberg',
  'Johann Aalberg',
  'Jussi Aalberg',
  'Jon Aalberg',
  'Yūḥannā Aalberg',
  'Evan Aalberg',
  'Jackie Aalberg',
  'Joan Aalberg',
  'Ioannes Aalberg',
  'Ioann Aalberg',
  'Yvan Aalberg',
  'Hannes Aalberg',
  'Jehannes Aalberg',
  'Joannes Aalberg',
  'João Aalberg',
  'Xoán Aalberg',
  'Ian Aalberg',
  'Ivashka Aalberg',
  'Gianni Aalberg',
  'Sean Aalberg',
  'Ioan Aalberg',
  'Jacky Aalberg',
  'Ivan Aalberg',
  'Jehan Aalberg',
  'Juhani Aalberg',
  'Iwan Aalberg',
  'Jaan Aalberg',
  'Yanka Aalberg',
  'Jón Aalberg',
  'Jóhan Aalberg',
  'Janka Aalberg',
  'Hanns Aalberg',
  'Iwo Aalberg',
  'Giannis Aalberg',
